## README 
Website: https://www.eqtlgen.org/phase1.html

Paper: https://www.nature.com/articles/s41588-021-00913-zhttps://www.nature.com/articles/s41588-021-00913-z

This Jupyter notebook README covers cis-eQTL and trans-eQTL results from the eQTLGen project. The dataset includes significant files for both cis-eQTL and trans-eQTL analyses. 

Files contain various columns with information on P-value, SNP rs ID, SNP chromosome, SNP position, assessed and not assessed alleles, Z-score, ENSG and HGNC names of eQTL genes, gene chromosome, gene position, number of cohorts, number of samples, false discovery rate, and Bonferroni-corrected P-value.

The cis-eQTL analysis includes 19,250 genes expressed in blood, with SNP-gene combinations within 1Mb from the gene center and tested in at least 2 cohorts. The trans-eQTL analysis tests 19,960 genes expressed in blood and 10,317 trait-associated SNPs based on GWAS Catalog, Immunobase, and Astle et al. study. Trans-eQTL combinations have a distance of >5Mb and were tested in at least 2 cohorts.

The FDR calculation uses a pruned set of SNPs for trans-eQTL mapping and permutation-based FDR calculation. Crossmapping filters are applied to identify and remove potential artifacts in trans-eQTL results, recalculating the FDR afterward. Note that the full results file has not been filtered for cross-mapping effects, which may lead to artifacts in the data.

The code below demonstrates the process of creating a graph-based representation of the combined cis and trans-eQTL data using PyTorch Geometric. This process can be broken down into several steps:

1. Combine cis and trans dataframes: The code begins by concatenating the cis and trans dataframes into a single dataframe named 'data', which contains information on both cis-eQTL and trans-eQTL results. This combined dataset simplifies the process of working with the data and ensures that all relevant information is contained within a single data structure.

2. Create mappings for genes and SNPs: To represent the genes and SNPs as nodes in the graph, integer indices are assigned to each unique gene and SNP. This is done using dictionaries called 'gene_to_idx' and 'snp_to_idx', which map the gene and SNP identifiers to their corresponding integer indices.

3. Generate node type labels: Node type labels are created using PyTorch tensors, distinguishing between gene nodes (assigned a label of 0) and SNP nodes (assigned a label of 1). This differentiation is useful for various graph-based analyses and machine learning tasks that require knowledge of node types.

4. Create edges based on gene and SNP indices: Edges in the graph represent the relationships between genes and SNPs. These edges are created by iterating over the 'data' dataframe and extracting the corresponding gene and SNP indices from the previously created mappings. The edges are then represented as a PyTorch tensor with a long data type.

5. Convert edges to undirected: Since the relationships between genes and SNPs are undirected, the edges in the graph should also be undirected. This is achieved using the 'to_undirected()' function from PyTorch Geometric, which ensures that the graph correctly represents the underlying biology.

6. Create a PyTorch Geometric graph: Finally, the graph is created using the PyTorch Geometric 'Data' class. The node types and edge indices are used as inputs to instantiate the graph object, which can then be utilized for further analysis and visualization.

The resulting 'graph' object is a PyTorch Geometric representation of the combined cis and trans-eQTL data. The prediction task is to predict new association edges given the training edges, with the task type being link prediction. Below are a few important graph statistics:

- Number of nodes: 3681495
- Number of SNP nodes: 3664025
- Number of Gene nodes: 17470
- Number of edges: 10567450
- Number of connected components: 424
- Average degree: 5.74
- Median degree: 2.0
- Standard deviation of degree: 69.81
- Density: 0.0000015594
- Assortativity: -0.2267915607

The prediction task is to predict new association edges given the training edges, with the task type being link prediction. The data splitting is random while maintaining an equal proportion of cis- and trans- associations.

## Data Setup 

### Libraries

In [2]:
import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pyarrow.parquet as pq


from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_fscore_support, roc_auc_score, average_precision_score
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler


import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.nn.init as init

import torch_sparse

import torch_geometric
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv, GATConv
from torch_geometric.utils import to_undirected, negative_sampling, subgraph
from torch_geometric.transforms import RandomLinkSplit

import networkx as nx
from ogb.io import DatasetSaver
from ogb.linkproppred import LinkPropPredDataset

In [3]:
print(f"PyTorch version: {torch.__version__}")
print(f"PyTorch Geometric version: {torch_geometric.__version__}")

PyTorch version: 2.0.0+cu118
PyTorch Geometric version: 2.3.1


In [4]:
if torch.cuda.is_available():
    device = torch.device("cuda")          # Current CUDA device
    print(f"Using {torch.cuda.get_device_name()} ({device})")
    print(f"CUDA version: {torch.version.cuda}")
    print(f"Number of CUDA devices: {torch.cuda.device_count()}")
else:
    print("CUDA is not available on this device.")

Using NVIDIA GeForce RTX 3060 Ti (cuda)
CUDA version: 11.8
Number of CUDA devices: 1


### Define gene mapping function

In [5]:
data_types = {'Pvalue': float, 'SNP': str, 'SNPChr': str, 'SNPPos': float,
              'AssessedAllele': str, 'OtherAllele': str, 'Zscore': float,
              'Gene': str, 'GeneSymbol': str, 'GeneChr': str, 'GenePos': float,
              'NrCohorts': int, 'NrSamples': int, 'FDR': float,
              'BonferroniP': float, 'GeneStart': float, 'GeneEnd': float}
gene_data = pd.read_csv("sig-combined-with-genes.csv", dtype=data_types)

def add_gene_start_end(df, gene_data):
    # Group gene_data by Gene
    gene_groups = gene_data.groupby('Gene')

    # Create a dictionary mapping genes to their GeneStart and GeneEnd values
    gene_dict = {
        gene: {'GeneStart': group['GeneStart'].iloc[0], 'GeneEnd': group['GeneEnd'].iloc[0]}
        for gene, group in gene_groups
        if not group['GeneStart'].isna().any() and not group['GeneEnd'].isna().any()
    }

    # Set GeneStart and GeneEnd columns of the dataframe using the gene_dict
    df = df.assign(
        GeneStart=df['Gene'].apply(lambda gene: gene_dict.get(gene, {}).get('GeneStart')),
        GeneEnd=df['Gene'].apply(lambda gene: gene_dict.get(gene, {}).get('GeneEnd'))
    )

    # Drop rows with missing GeneStart or GeneEnd values
    df.dropna(subset=['GeneStart', 'GeneEnd'], inplace=True)

    return df

### Load files

In [6]:
%%time
# Read the Parquet files into Pandas DataFrames
trans = pd.read_parquet('big-trans.parquet')
cis = pd.read_parquet('big-cis.parquet')

sig_trans = pd.read_parquet('sig-trans.parquet')
sig_cis = pd.read_parquet('sig-cis.parquet')

CPU times: total: 6min 8s
Wall time: 7min 15s


In [7]:
# Print DataFrame lengths
print("trans length:", len(trans))
print("cis length:", len(cis))
print("sig_trans length:", len(sig_trans))
print("sig_cis length:", len(sig_cis))

trans length: 203547746
cis length: 127341798
sig_trans length: 59786
sig_cis length: 10507664


In [8]:
%%time
trans_sample = trans.sample(frac=0.25, random_state=42)
cis_sample = cis.sample(frac=0.25, random_state=42)

CPU times: total: 44.7 s
Wall time: 6min 3s


### Filter by Sig

In [9]:
%%time
# For trans_sample DataFrame
trans_sample['Sig'] = trans_sample.apply(lambda row: 1 if row.name in sig_trans.index and row.equals(sig_trans.loc[row.name]) else 0, axis=1)

# For cis_sample DataFrame
cis_sample['Sig'] = cis_sample.apply(lambda row: 1 if row.name in sig_cis.index and row.equals(sig_cis.loc[row.name]) else 0, axis=1)


CPU times: total: 3min 29s
Wall time: 7min 38s


### Map gene start/end info to dfs

In [10]:
%%time
trans_sample = add_gene_start_end(trans_sample, gene_data)
cis_sample = add_gene_start_end(cis_sample, gene_data)

CPU times: total: 28.3 s
Wall time: 1min 28s


In [16]:
print(cis_sample.columns)
print(trans_sample.columns)

Index(['Pvalue', 'SNP', 'SNPChr', 'SNPPos', 'AssessedAllele', 'OtherAllele',
       'Zscore', 'Gene', 'GeneSymbol', 'GeneChr', 'GenePos', 'NrCohorts',
       'NrSamples', 'FDR', 'BonferroniP', 'Sig', 'GeneStart', 'GeneEnd'],
      dtype='object')
Index(['Pvalue', 'SNP', 'SNPChr', 'SNPPos', 'AssessedAllele', 'OtherAllele',
       'Zscore', 'Gene', 'GeneSymbol', 'GeneChr', 'GenePos', 'NrCohorts',
       'NrSamples', 'FDR', 'BonferroniP', 'Sig', 'GeneStart', 'GeneEnd'],
      dtype='object')


In [15]:
print(len(cis_sample))
print(len(trans_sample))
print(len(cis))
print(len(trans))

22914564
34537653
127341798
203547746


### To CSV

In [17]:
%%time
trans_sample.to_csv('trans-sample-genes.csv')
cis_sample.to_csv('cis-sample-genes.csv')

CPU times: total: 4min 3s
Wall time: 5min 33s
